# Challenge 1

In [949]:
import re

In [950]:
with open("./data/data_d7.txt") as f:
    lines = f.readlines()

for i in range(0, len(lines)):
    lines[i] = lines[i].replace("\n", "")

In [951]:
class Directory:
    def __init__(self, name: str):
        self.name = name
        self.subDirs = {}
        self.files = []
        self.size = 0
        self.totalSize = 0
        
    def __str__(self):
        return (str({"name": self.name,
                     "subDirs": self.subDirs,
                     "files": self.files,
                     "size": self.size,
                     "totalSize": self.totalSize}))
        
    def addDir(self, name: str):
        self.subDirs[name] = Directory(name)
        
    def findDir(self, name: str):
        return self.subDirs.get(name, None)
        
    def addFile(self, name: str, size: int):
        self.files += [(size, name)]
        
    def calculateSize(self):
        self.size = sum([size for size, _ in self.files])
    
    def setTotalSize(self, size: int):
        self.totalSize = size

In [952]:
fileSystem = {}
currPath = []
for line in lines:
    if line.split()[0] == "$" and line.split()[1] == "cd" and line.split()[2] != "..":
        currPath += [line.split()[2]]
        fileSystem["_".join(currPath)] = []
    elif line.split()[0] == "$" and line.split()[1] == "cd":
        currPath.pop()
    elif re.search("[0-9]", line.split()[0]):
        fileSystem["_".join(currPath)] += [(int(line.split()[0]), line.split()[1])]

In [953]:
folders = Directory("/")
for file in fileSystem["/"]:
    folders.addFile(file[1], file[0])
folders.size = sum([size for size, _ in folders.files])

In [954]:
for key, value in fileSystem.items():
    if key == "/":
        pass
    
    currentFolder = folders
    for subFolder in key.split("_")[1:]:
        if currentFolder.findDir(subFolder):
            currentFolder = currentFolder.findDir(subFolder)
        else:
            currentFolder.addDir(subFolder)
            currentFolder = currentFolder.findDir(subFolder)
            for file in value:
                currentFolder.addFile(file[1], file[0])
                currentFolder.calculateSize()

In [955]:
def calculateTotalSize(folders):
    if len(folders.subDirs) == 0:
        folders.setTotalSize(folders.size)
        return folders.totalSize
    
    sum_ = folders.size
    queue = [folder for folder in folders.subDirs.values()]
    while queue:
        folder = queue.pop(0)
        sum_ += calculateTotalSize(folder)
    folders.setTotalSize(sum_)
        
    return sum_

In [957]:
def calculateChAnswer(folders):
    if len(folders.subDirs) == 0:
        return 0
    
    sum_ = 0
    queue = [folder for folder in folders.subDirs.values()]
    while queue:
        folder = queue.pop(0)
        if folder.totalSize <= 100000:
            sum_ += folder.totalSize + calculateChAnswer(folder)
        else:
            sum_ += calculateChAnswer(folder)
            
    return sum_
    

In [958]:
print(calculateChAnswer(folders))

1297683


# Challenge 2

In [989]:
neededSpace = 30000000 - (70000000 - folders.totalSize)

In [990]:
def findLowestToDelte(folders):
    if len(folders.subDirs) == 0:
        if folders.totalSize >= neededSpace:
            return [folders.totalSize]
    
    candidates = []
    if folders.totalSize >= neededSpace:
        candidates.append(folders.totalSize)
        
    queue = [folder for folder in folders.subDirs.values()]
    while queue:
        folder = queue.pop(0)
        candidates += findLowestToDelte(folder)
        
    return candidates

In [991]:
min(findLowestToDelte(folders))

5756764

# A really great solution from reddit

In [1006]:
from collections import defaultdict
from itertools import accumulate
from operator import concat

total_space = 70000000
required_space = 30000000
max_size = 100000
curr_dir_path = list()
dir_tree = {}


def keep_useful_info(entry):
    # keep only directory changes and file size
    return (True, entry[1]) if entry[0] == "$ cd" else (False, int(entry[0])) if entry[0][0].isdigit() else None


with open("./data/data_d7.txt", 'r') as file:
    commands = filter(bool, map(keep_useful_info, (line.strip('\n').rsplit(' ', 1) for line in file)))
    while True:
        try:
            change_dir, info = next(commands)
            if change_dir:
                curr_dir_path.pop() if info == ".." else curr_dir_path.append(info)
            else:
                for directory in accumulate(curr_dir_path, concat):  # the name of a directory is its path
                    dir_tree[directory] = dir_tree.get(directory, 0) + info
        except StopIteration:
            break

# PART ONE
print(sum(filter(lambda x: x <= max_size, dir_tree.values())))

# PART TWO
to_del_space = required_space - (total_space - dir_tree["/"])  # required space - free space
print(to_del_space + min(filter(lambda x: x >= 0, map(lambda dir_size: dir_size - to_del_space, dir_tree.values()))))

1297683
5756764


In [1014]:
# Parse the input and extract the list of directories and files
directories = []
files = []

# Read the input
input_string = "./data/data_d7.txt"

# Split the input into lines
lines = input_string.split("\n")

# Parse the lines
for line in lines:
  # Split the line into words
  words = line.split(" ")

  # If the line starts with "cd", it is a directory
  if words[0] == "cd":
    directories.append(words[1])
  # Otherwise, it is a file
  else:
    files.append((words[1], int(words[0])))

# Initialize the sum of sizes of directories with a total size of at most 100000
result = 0

# Initialize the list of visited directories
visited_directories = []

# Iterate over the directories and calculate their sizes
for directory in directories:
  # Check if the directory has already been visited
  if directory in visited_directories:
    continue

  # Initialize the size of the directory
  directory_size = 0

  # Iterate over the files and add the sizes of the files in the directory
  for file in files:
    # Extract the name and size of the file
    file_name, file_size = file

    # Check if the file is in the current directory
    if file_name.startswith(directory):
      # Add the size of the file to the size of the directory
      directory_size += file_size

  # Add the directory to the list of visited directories
  visited_directories.append(directory)

  # Check if the size of the directory is at most 100000
  if directory_size <= 100000:
    # If it is, add its size to the final result
    result += directory_size

# Print the final result
print(result)

IndexError: list index out of range